In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

import cantools
import can
from pprint import pprint

In [2]:
#Load dataset from .txt file
Attack_ds = pd.read_csv('Fuzzy_dataset_KIA_SOUL.csv',
                          names=['Timestamp','ID','DLC','allData','Flag'], 
                          sep=',', low_memory=False)

#No. of rows in the dataset
rowCount = Attack_ds.count()[0]
print('No. of rows in dataset: ', rowCount)

No. of rows in dataset:  245238


In [3]:
Attack_ds_filtered = Attack_ds[Attack_ds['ID'] == '220']
Attack_ds_filtered.loc[Attack_ds_filtered['Flag'] == 'R', 'Flag'] = 0
Attack_ds_filtered.loc[Attack_ds_filtered['Flag'] == 'T', 'Flag'] = 1

Attack_ds_filtered.reset_index(drop=True, inplace=True)

,Timestamp,ID,DLC,allData,Flag
0,1513925133,220,8,F0 03 F8 03 09 00 42 10,0
1,1513925133,220,8,EF 03 F4 03 09 00 45 10,0
2,1513925133,220,8,EC 03 F2 03 09 00 46 10,0
3,1513925133,220,8,EA 03 F4 03 09 00 43 10,0
4,1513925133,220,8,EC 03 F6 03 09 00 41 10,0
5,1513925133,220,8,EE 03 F9 03 0C 00 3E 10,0
6,1513925133,220,8,F0 03 FB 03 09 00 40 10,0
7,1513925133,220,8,F1 03 F9 03 0C 00 43 10,0
8,1513925133,220,8,F1 03 F4 03 09 00 46 10,0
9,1513925133,220,8,EE 03 F2 03 09 00 49 10,0


In [4]:
#No. of rows in the dataset
rowCount = Attack_ds_filtered.count()[0]
print('No. of rows in dataset: ', rowCount)

No. of rows in dataset:  18616


In [5]:
for idx in range(rowCount):
    for j in range(Attack_ds_filtered['DLC'][idx]+1):
        if j == 0:
            allData = Attack_ds_filtered['allData'][idx][3*j:2+3*j]
        elif j == Attack_ds_filtered['DLC'][idx]:
            Attack_ds_filtered.loc[idx,'allData'] = allData
        else:
            allData = allData + Attack_ds_filtered['allData'][idx][3*j:2+3*j]

In [7]:
#Display Flag options
print('###############################################')
print('Flags listed in the dataset: ')
display(Attack_ds_filtered.groupby(['Flag'])['Flag'].count())

###############################################
Flags listed in the dataset: 


Flag
0    16003
1     2613
Name: Flag, dtype: int64

In [8]:
def merge_dicts(dict1, dict2):
    return(dict1.update(dict2))

def create_atk_ds(atk_ds, ID):
    global msg_count, nonAtk_msg, Atk_msg, error
    msg_count = 0
    nonAtk_msg = 0
    Atk_msg = 0
    error = 0
    LOG_EVERY_N = 1000

    dbc = cantools.database.load_file(r'C:\Users\sabbi\Desktop\QCProject\opendbc\hyundai_kia_generic.dbc')
    
    for idx, row in atk_ds.iterrows():
        if row['ID'] == ID:
            try:
                temp_dict = {'Attack':row['Flag'],'Timestamp':row['Timestamp']
                             #'ID':row['ID'], #'RTR':row['RTR'],
                             #'DLC':row['DLC']
                            }
            
                raw_data = row['allData']
                decoded_data = dbc.decode_message(0x220, bytes.fromhex(raw_data))
                merge_dicts(temp_dict,decoded_data)

                #print('Row {} contains data: {}'.format(idx,temp_dict))
                #print('Timestamp: {}, Raw data: {}'.format(row['Timestamp'], raw_data))
                
                if msg_count == 0:
                    Processed_ds = pd.DataFrame(temp_dict, index = [0])
                else:
                    ds_row = pd.DataFrame(temp_dict, index = [0])
                    Processed_ds = pd.concat([Processed_ds, ds_row], ignore_index=True)
                
                
                if row['Flag'] == 1:
                    Atk_msg += 1
                else:
                    nonAtk_msg += 1
                
                msg_count += 1
                
                '''
                if msg_count == 10:
                    break
                '''
            except:
                error += 1
                
            if msg_count and msg_count % LOG_EVERY_N == 0:
                print(f"logging : {msg_count}")
                
                
    print('{} CAN messages decoded successfully!'.format(msg_count))
    print('{} CAN messages were injected and {} CAN messages were authentic!'.format(Atk_msg, nonAtk_msg))
    print('{} CAN messages could not be decoded!'.format(error))
    
    return Processed_ds

In [9]:
Processed_ds = create_atk_ds(Attack_ds_filtered, ID = '###')

logging : 1000
logging : 2000
logging : 3000
logging : 4000
logging : 5000
logging : 6000
logging : 7000
logging : 8000
logging : 9000
logging : 10000
logging : 11000
logging : 12000
logging : 13000
logging : 14000
logging : 15000
logging : 16000
logging : 17000
logging : 18000
18616 CAN messages decoded successfully!
2613 CAN messages were injected and 16003 CAN messages were authentic!
0 CAN messages could not be decoded!


In [10]:
#No. of rows in the dataset
rowCount = Processed_ds.count()[0]
print('No. of rows in dataset: ', rowCount)

Processed_ds_copy = Processed_ds
Processed_ds.head(10)

No. of rows in dataset:  18616


,Attack,Timestamp,LAT_ACCEL,LAT_ACCEL_STAT,LAT_ACCEL_DIAG,LONG_ACCEL,LONG_ACCEL_STAT,LONG_ACCEL_DIAG,CYL_PRES,CYL_PRES_STAT,CYL_PRESS_DIAG,YAW_RATE,YAW_RATE_STAT,YAW_RATE_DIAG,ESP12_Checksum,ESP12_AliveCounter
0,0,1513925133,-0.15,0,0,9.61,1,1,57.6,0,0,-35.83,0,1,0,1
1,0,1513925133,-0.16,0,0,9.29,1,1,57.6,0,0,-28.15,0,1,0,1
2,0,1513925133,-0.19,0,0,9.13,1,1,57.6,0,0,-25.59,0,1,0,1
3,0,1513925133,-0.21,0,0,9.29,1,1,57.6,0,0,-33.27,0,1,0,1
4,0,1513925133,-0.19,0,0,9.45,1,1,57.6,0,0,-38.39,0,1,0,1
5,0,1513925133,-0.17,0,0,9.69,1,1,76.8,0,0,35.85,1,0,0,1
6,0,1513925133,-0.15,0,0,9.85,1,1,57.6,0,0,-40.95,0,1,0,1
7,0,1513925133,-0.14,0,0,9.69,1,1,76.8,0,0,-33.27,0,1,0,1
8,0,1513925133,-0.14,0,0,9.29,1,1,57.6,0,0,-25.59,0,1,0,1
9,0,1513925133,-0.17,0,0,9.13,1,1,57.6,0,0,-17.91,0,1,0,1


In [14]:
with pd.ExcelWriter('C:/Users/sabbi/Desktop/QCProject/Processed_data/Kia Soul Fuzzy/KiaSoulFuzzy_ID_220.xlsx') as writer:
    Processed_ds.to_excel(writer, index = False) 